In [4]:
import pinecone
import openai
import numpy as np
import os
from dotenv import load_dotenv
# Load variables from the .env file
load_dotenv('./Sn33k/.env')

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENVIRONMENT= os.getenv("PINECONE_ENVIRONMENT")

openai.api_key = OPENAI_API_KEY


# Pinecone Init

In [23]:
pinecone.init(api_key=OPENAI_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone.list_indexes()

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'www-authenticate': 'API key is missing or invalid for the environment "us-west1-gcp-free". Check that the correct environment is specified.', 'content-length': '119', 'date': 'Mon, 03 Jul 2023 09:30:29 GMT', 'server': 'envoy'})
HTTP response body: API key is missing or invalid for the environment "us-west1-gcp-free". Check that the correct environment is specified.


## String Query embeddings

In [12]:
def get_text_embedding(text):
    response = openai.Embedding.create(input=text, engine="text-embedding-ada-002")
    embedding = response["data"][0]["embedding"]
    return embedding

Test

In [13]:
query_text = "What is Singlife"
query_vector = get_text_embedding(query_text)
top_k = 5  # Replace with the desired number of nearest neighbors

results = index.query(queries=[query_vector], top_k=top_k)

PineconeProtocolError: Failed to connect; did you specify the correct index name?